In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
from datetime import datetime
import numpy as np
# Import the API keys.
from config import api_keys
# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy

In [25]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
want_rain = input("Do you want it to be raining? (yes/no) ").lower()
want_snow = input("Do you want it to be snowing? (yes/no) ").lower()


ValueError: could not convert string to float: 

In [27]:
city_csv = './data/WeatherPy_database.csv'
city_data_df = pd.read_csv(city_csv)
city_data_df.head()
city_data_df.dtypes


70.0 90.0 no no


In [17]:
# Filter the dataset to find the cities that fit the criteria.

# checking for temp range
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)].dropna()


In [5]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng","Current Description"]].copy()
hotel_df["Hotel Name"] = ""

In [6]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": api_keys['g_key']
}

# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        print(f'Checking at location {hotels["results"][0]["name"]}...')
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except:
        print('Hotel not found...skipping')
        continue



Checking at location ARTIEM Capri...
Checking at location Casa Tia Villa...
Hotel not found...skipping
Checking at location Isles Sunset Lodge...
Checking at location Guest Hotel...
Checking at location Hotel Arena Blanca...
Checking at location The Esplanade Hotel...
Checking at location Vila Nova Hotel...
Checking at location El Medina...
Checking at location Hilo Hawaiian Hotel...
Checking at location Paradise Inn...
Checking at location Derogongan Residence...
Checking at location Hotel Tesoro Los Cabos...
Checking at location Osprey Beach Hotel...
Checking at location Gostevoy Dom Terem...
Checking at location Complexo Sonho Real...
Checking at location Veranda Paul et Virginie Hotel & Spa...
Checking at location Aldea del Bazar Hotel and Spa...
Checking at location Nanning Travelling With Hostel...
Checking at location Ranau Backpackers Hostel...
Checking at location Hotel Marrakech...
Checking at location Gîte kajeon...
Checking at location Chambres d'hôte "La Trinité"...
Checki

In [28]:
hotel_df


,City,Country,Max Temp,Lat,Lng,Current Description,Hotel Name
1,Mahon,ES,82.40,39.89,4.27,overcast clouds,ARTIEM Capri
2,Bambous Virieux,MU,75.99,-20.34,57.76,scattered clouds,Casa Tia Villa
3,Albany,US,73.00,42.60,-73.97,few clouds,
6,Butaritari,KI,82.90,3.07,172.79,light rain,Isles Sunset Lodge
11,Manggar,ID,79.38,-2.88,108.27,light rain,Guest Hotel
...,...,...,...,...,...,...,...
560,Caraballeda,VE,70.00,10.61,-66.85,few clouds,Hotel 15 Letras
566,Svetlogorsk,BY,72.99,52.63,29.73,scattered clouds,Hotel Svetlogorsk JBIK
567,Neiafu,TO,76.98,-18.65,-173.98,scattered clouds,Mystic Sands
568,Saint-Georges,GD,78.80,12.06,-61.75,scattered clouds,Radisson Grenada Beach Resort


In [8]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_keys['g_key'])

In [24]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
info_box_template = """
<dl>
<dt><strong>Hotel Name</strong></dt><dd>{Hotel Name}</dd>
<dt><strong>City</strong></dt><dd>{City}</dd>
<dt><strong>Country</strong></dt><dd>{Country}</dd>
<dt><strong>Current Weather</strong></dt><dd>{Current Description} {Max Temp} °F</dd>
</dl>
"""
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(35.0, 40.0), zoom_level=1.7)
marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig


Figure(layout=FigureLayout(height='420px'))

In [29]:
# Create the output file (CSV).
output_data_file = './data/WeatherPy_vacation.csv'
# Export the City_Data into a CSV.
hotel_df.to_csv(output_data_file, index_label="City_ID")